In [2]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("./datasets/datathon_train.csv")

def get_time(string):
  return int(string[:2])
dataset['DEP_TIME'] = dataset['DEP_TIME_BLK'].apply(get_time)



In [3]:
dataset_clean = dataset.drop(['Id', 'ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'DEP_TIME_BLK'], axis = 1)
dataset_clean.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEPARTING_AIRPORT,DEST,DISTANCE,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,MANUFACTURE_YEAR,NUMBER_OF_SEATS,...,PLANE_AGE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND,DEP_DELAY_NEW,IS_DELAYED,DEP_TIME
0,1,17,4,Raleigh-Durham International,JFK,427,6,13,2014.0,76,...,5,Ronald Reagan Washington National,0.01,0.0,0.0,49.0,4.70,2.0,1,18
1,2,27,3,San Jose International,BOS,2689,2,3,2002.0,162,...,17,John F. Kennedy International,0.21,0.0,0.0,64.0,12.75,0.0,0,22
2,8,14,3,Los Angeles International,ATL,1947,1,30,1996.0,199,...,23,NONE,0.00,0.0,0.0,74.0,8.50,0.0,0,6
3,3,16,6,Newark Liberty International,DTW,488,3,23,2015.0,76,...,4,Dallas Fort Worth Regional,0.00,0.0,0.0,56.0,17.67,0.0,0,14
4,6,20,4,Douglas Municipal,JFK,541,4,24,1998.0,128,...,21,Myrtle Beach International,0.19,0.0,0.0,89.0,9.17,79.0,1,15


In [4]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Assuming 'X_df' is your dataset and 'Departing Airport Name' is a categorical column
categorical_columns = ['PREVIOUS_AIRPORT', 'CARRIER_NAME', 'DEPARTING_AIRPORT', 'DEST']

# Apply one-hot encoding to categorical columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(dataset_clean[categorical_columns])

# Drop the original categorical columns from the original DataFrame
dataset_clean = dataset_clean.drop(categorical_columns, axis=1)

# Convert the one-hot encoded NumPy array back to a DataFrame with feature names
encoded_feature_names = []
for category, column in zip(encoder.categories_, categorical_columns):
    encoded_feature_names.extend([f"{column}_{cat}" for cat in category])
X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_feature_names)

# Concatenate the one-hot encoded columns with the original dataset
X_df = pd.concat([dataset_clean, X_encoded_df], axis=1)


/home/sae/ENTER/envs/newconda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
# Specify the file path where you want to save the data
file_path = "data_output.txt"

X_df_10 = X_df.head()
# Write the DataFrame to the text file
X_df_10.to_csv(file_path, sep='\t', index=False)

In [6]:
y = X_df['IS_DELAYED'].to_numpy()
New_X_df = X_df.drop(['DEP_DELAY_NEW', 'IS_DELAYED'], axis = 1)
X = New_X_df.to_numpy()

In [7]:
# Display the column names of the feature DataFrame
feature_columns = New_X_df.columns
print("Feature Columns:")
print(feature_columns)

Feature Columns:
Index(['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DISTANCE', 'SEGMENT_NUMBER',
       'CONCURRENT_FLIGHTS', 'MANUFACTURE_YEAR', 'NUMBER_OF_SEATS',
       'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH',
       ...
       'DEST_TYR', 'DEST_TYS', 'DEST_UIN', 'DEST_VEL', 'DEST_VLD', 'DEST_VPS',
       'DEST_WYS', 'DEST_XNA', 'DEST_XWA', 'DEST_YUM'],
      dtype='object', length=812)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [9]:
#Create and train a decision tree classifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42)  # You can customize hyperparameters if needed
clf.fit(X_train, y_train)

# Step 4: Evaluate the classifier's performance on the testing data
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.5960055936032127
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.66      0.66     83870
           1       0.49      0.50      0.49     55575

    accuracy                           0.60    139445
   macro avg       0.58      0.58      0.58    139445
weighted avg       0.60      0.60      0.60    139445



Accuracy: 0.5960055936032127
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.66      0.66     83870
           1       0.49      0.50      0.49     55575

    accuracy                           0.60    139445
   macro avg       0.58      0.58      0.58    139445
weighted avg       0.60      0.60      0.60    139445

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 20, 50, 100],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

: 

: 